# Imports & prerequisites

In [3]:
!cp /content/drive/MyDrive/Colab\ Notebooks/include/data_utils.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/tuning_utils.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/models.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/sca_utils.py /content/
!pip install keras-tuner --upgrade

In [4]:
import os, h5py
import numpy as np
from data_utils import SyntheticDataLoader, HdfToTfrAdapter, TfrDataLoader
from tuning_utils import CleanMemoryTuner, TensorBoard, EarlyStopping, vgg_cbam_synthetic_builder
from models import vgg_cnn_cbam

# Hyperparameter tuning

On a random dataset (where the plaintext distribution is fully random), perform the hyperparameter search in order to figure out the best architectures for CNN with SE and CBAM modules.

## SE tuning

Perform the tuning on 16 bytes of the CNN Squeeze-and-Excitation network.

_Notes_:
* `model_builder_synthetic_se_cnn` function for model building

In [ ]:
# --- SE Configuration ---

TUNER_DIRECTORY = "/content/drive/MyDrive/mbedtls_synthetic_traces/hyperparameter_tuning_efficient"
TFRECORD_DIR = "/content/drive/MyDrive/mbedtls_synthetic_traces/profiling_windows/tfrecord_data"
BYTES_TO_TUNE = [15]
INPUT_DIM = 701
NUM_SAMPLES_PER_BYTE = 50001
BATCH_SIZE = 128
EPOCHS_PER_TRIAL = 25
MAX_TRIALS = 10
EXECUTIONS_PER_TRIAL = 1

In [ ]:
tfr_data_loader = TfrDataLoader()

In [ ]:
for target_byte in BYTES_TO_TUNE:
    print(f"\n\n{'='*60}")
    print(f"--- STARTING EFFICIENT HYPERPARAMETER SEARCH FOR BYTE {target_byte} ---")
    print(f"{'='*60}\n")

    TFRECORD_FILE_PATH = os.path.join(TFRECORD_DIR, f'byte_{target_byte}.tfrecord')
    if not os.path.exists(TFRECORD_FILE_PATH):
        raise FileNotFoundError(f"TFRecord file not found: {TFRECORD_FILE_PATH}. Please run the conversion script first.")


    ds_train_for_tuner, ds_val_for_tuner, ds_val_full_for_ranking, _, _ = tfr_data_loader.create_dataset_from_tfrecord(
        tfrecord_path=TFRECORD_FILE_PATH,
        input_dim=INPUT_DIM,
        batch_size=BATCH_SIZE,
        val_split=0.2,
        num_samples=NUM_SAMPLES_PER_BYTE
    )

    tuner = CleanMemoryTuner(
        hypermodel=lambda hp: model_builder_synthetic_se_cnn(
            hp,
            model_fun=cnn_squeeze_excite,
            input_dim=INPUT_DIM
        ),
        objective="val_accuracy",
        max_trials=MAX_TRIALS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        directory=TUNER_DIRECTORY,
        project_name=f"tuning_byte_{target_byte}",
        overwrite=True
    )

    tuner.search_space_summary()

    log_dir = os.path.join(TUNER_DIRECTORY, f"tb_logs_byte_{target_byte}")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\n--- Running search for byte {target_byte} with efficient pipeline ---")
    tuner.search(
        ds_train_for_tuner,
        validation_data=ds_val_for_tuner,
        epochs=EPOCHS_PER_TRIAL,
        callbacks=[early_stop, tensorboard_callback]
    )

    print(f"\n--- SEARCH COMPLETE FOR BYTE {target_byte} ---")
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"""
    Best hyperparameters for byte {target_byte}:
    - Filter Scale: {best_hps.get('filter_scale')}
    - Dropout Rate: {best_hps.get('dropout'):.4f}
    - SE Ratio: {best_hps.get('se_ratio')}
    - Batch Normalization: {best_hps.get('batch_normalize')}
    - Optimizer: {best_hps.get('optimizer')}
    - Learning Rate: {best_hps.get('learning_rate')}
    """)

---

## CBAM tuning

Perform the tuning on 16 bytes of the CNN Convolutional Block Attention Module network.

_Notes_:
* `vgg_cbam_synthetic_builder` function for model building

In [5]:
# --- CBAM Configuration ---

TUNER_DIRECTORY = "/content/drive/MyDrive/mbedtls_synthetic_traces/RANDOM_datasets/CBAM/hypertuning"
TFRECORD_DIR = "/content/drive/MyDrive/mbedtls_synthetic_traces/RANDOM_datasets/profiling_windows/tfrecord_data"
BYTES_TO_TUNE = [2]
INPUT_DIM = 701
NUM_SAMPLES_PER_BYTE = 50001
BATCH_SIZE = 128
EPOCHS_PER_TRIAL = 25
MAX_TRIALS = 10
EXECUTIONS_PER_TRIAL = 1

In [6]:
tfr_data_loader = TfrDataLoader()

In [7]:
for target_byte in BYTES_TO_TUNE:
    print(f"\n\n{'='*60}")
    print(f"--- STARTING EFFICIENT HYPERPARAMETER SEARCH FOR BYTE {target_byte} ---")
    print(f"{'='*60}\n")

    TFRECORD_FILE_PATH = os.path.join(TFRECORD_DIR, f'byte_{target_byte}.tfrecord')
    if not os.path.exists(TFRECORD_FILE_PATH):
        raise FileNotFoundError(f"TFRecord file not found: {TFRECORD_FILE_PATH}. Please run the conversion script first.")


    ds_train_for_tuner, ds_val_for_tuner, ds_val_full_for_ranking, _, _ = tfr_data_loader.create_dataset_from_tfrecord(
        tfrecord_path=TFRECORD_FILE_PATH,
        input_dim=INPUT_DIM,
        batch_size=BATCH_SIZE,
        val_split=0.2,
        num_samples=NUM_SAMPLES_PER_BYTE
    )

    tuner = CleanMemoryTuner(
        hypermodel=lambda hp: vgg_cbam_synthetic_builder(
            hp,
            model_fun=vgg_cnn_cbam,
            input_dim=INPUT_DIM
        ),
        objective="val_accuracy",
        max_trials=MAX_TRIALS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        directory=TUNER_DIRECTORY,
        project_name=f"tuning_byte_{target_byte}",
        overwrite=True
    )

    tuner.search_space_summary()

    log_dir = os.path.join(TUNER_DIRECTORY, f"tb_logs_byte_{target_byte}")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    print(f"\n--- Running search for byte {target_byte} with efficient pipeline ---")
    tuner.search(
        ds_train_for_tuner,
        validation_data=ds_val_for_tuner,
        epochs=EPOCHS_PER_TRIAL,
        callbacks=[early_stop, tensorboard_callback]
    )

    print(f"\n--- SEARCH COMPLETE FOR BYTE {target_byte} ---")
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"""
    Best hyperparameters for byte {target_byte}:
    - Filter Scale: {best_hps.get('filter_scale')}
    - Dropout Rate: {best_hps.get('dropout'):.4f}
    - CBAM Ratio: {best_hps.get('cbam_ratio')}
    - Batch Normalization: {best_hps.get('batch_normalize')}
    - Optimizer: {best_hps.get('optimizer')}
    - Learning Rate: {best_hps.get('learning_rate')}
    """)

Trial 10 Complete [00h 17m 36s]
val_accuracy: 0.991599977016449

Best val_accuracy So Far: 1.0
Total elapsed time: 01h 00m 19s

--- SEARCH COMPLETE FOR BYTE 2 ---

    Best hyperparameters for byte 2:
    - Filter Scale: 1.5
    - Dropout Rate: 0.3000
    - CBAM Ratio: 32
    - Batch Normalization: True
    - Optimizer: adam
    - Learning Rate: 5e-05
    
